In [22]:
import json

In [23]:
import json
ENTITIES_FILEPATH='camera_trainingset/camera-ENTITIES.json'
JSON_FILES_FILEPATH = 'camera_trainingset/camera-JSON_FILES.json'
with open(ENTITIES_FILEPATH) as json_file:
    matching = json.load(json_file)
with open(JSON_FILES_FILEPATH) as json_file:
    json_filepaths = json.load(json_file)

In [24]:
len(json_filepaths)

29787

In [25]:
import os

filepaths = {}
DATASET_FILEPATH = 'camera_dataset/2013_camera_dataset/'
for filepath in json_filepaths:
    json_id = filepath['resource_id']
    domain = filepath['source_name']
    filename = str(filepath['json_number'])+'.json'
    filepaths[json_id]= os.path.join(DATASET_FILEPATH,domain,filename)

In [26]:
# positives samples è una tupla record1,record2,matching/nomatching
positives_samples = []
for entity in matching:
    instances = entity['instances']
    i=0
    for instance in instances:
        filepath = filepaths[instance]
        with open(filepath) as json_file:
            record = json.load(json_file)
            left_page_title = record['<page title>']
        for j in range(i+1,len(instances)):
            filepath = filepaths[instances[j]]
            with open(filepath) as json_file:
                record = json.load(json_file)
                right_page_title = record['<page title>']
                positives_samples.append((left_page_title,right_page_title,1))
        i += 1

In [130]:
positives_samples[200][2]

1

In [28]:
all_sources = os.listdir('camera_dataset/2013_camera_dataset')

In [29]:
import random
random.choice(all_sources)

'www.gosale.com'

In [30]:
import numpy as np
GLOVE_DIR='glove'
GLOVE_FILENAME='glove.840B.300d.txt'
wordToEmbeddingMap = {}
with open(GLOVE_DIR+'/'+GLOVE_FILENAME) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, 'f', sep=' ')
        if len(coefs) != 0:
            wordToEmbeddingMap[word] = coefs

In [31]:
import nltk
stopwords = nltk.download("stopwords")

from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
tokenizer = RegexpTokenizer(r'\w+')
stop_words = set(stopwords.words('english'))
stop_words.update(['.', ',', '"', "'", ':', ';', '(', ')', '[', ']', '{', '}','|','-'])

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jgmathew/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [32]:
BASE_DIR ='camera_dataset/2013_camera_dataset/'
source1 = random.choice(all_sources)
source2 = random.choice(all_sources)
while(source2==source1):
    source2 = random.choice(all_sources)
all_files_source1 = os.listdir(BASE_DIR+source1)
all_files_source2 = os.listdir(BASE_DIR+source2)

In [33]:
all_json_source1 = list(filter(lambda s: s.endswith('.json'), all_files_source1))
all_json_source2 = list(filter(lambda s: s.endswith('.json'),all_files_source2))

In [34]:
len(all_files_source1)

1142

In [35]:
len(all_files_source2)

15944

In [36]:
len(all_json_source1)

571

In [37]:
len(all_json_source2)

7972

In [38]:
def extract_titles(filenamelist,source,basedir=BASE_DIR):
    page_titles = []
    for filename in filenamelist:
        filepath = os.path.join(BASE_DIR,source,filename)
        with open(filepath) as json_file:
            json_ob = json.load(json_file)
            page_title = json_ob['<page title>']
            page_titles.append(page_title)
    return page_titles                

In [39]:
def createDistributedRapresentation(recordTitleList):
    embeddingMap = {}
    for recordtitle in recordTitleList:
        currentTitle = recordtitle
        tokens_title = tokenizer.tokenize(currentTitle)
        filtered_title = [word for word in tokens_title if word not in stop_words]
        #processo di creazione del vettore di embedding
        numeratoreVec = np.zeros(300)
        ntokens = 0
        for token in filtered_title:
            ntokens += 1
            embeddingVector = wordToEmbeddingMap.get(token)
            if embeddingVector is not None:
                numeratoreVec += embeddingVector
        tupleEmbedding = numeratoreVec /ntokens
        embeddingMap[currentTitle]= tupleEmbedding
    return embeddingMap

In [40]:
titlesList_source1 = extract_titles(all_json_source1,source1)

In [41]:
titlesList_source2 = extract_titles(all_json_source2,source2)

In [42]:
embedding_source1 = createDistributedRapresentation(titlesList_source1)
embedding_source2 = createDistributedRapresentation(titlesList_source2)

In [57]:
from sklearn.metrics.pairwise import cosine_similarity

embedding55 = list(embedding_source1.values())[0].reshape(1,-1)
embedding273 = list(embedding_source2.values())[100].reshape(1,-1)
cosineSim = cosine_similarity(embedding55,embedding273)
cosineSim[0][0]

0.49004589890647005

In [58]:
len(embedding_source1)

571

In [73]:
len(titlesList_source1)

571

In [76]:
for i in range(len(titlesList_source2)):
    for j in range(i+1, len(titlesList_source2)):
        if titlesList_source2[i] == titlesList_source2[j]:
            print('found duplicate records')
            print(i)
            print(j)
            print(titlesList_source2[i])
            print()
            print()

found duplicate records
15
3906
2.0 Megapixel Wireless Wifi 1920 X 1080p Hd Network Ip Cctv Security Camera - Buy Hd Network Ip Cctv Security Camera,Outdoor Wireless Wifi Hd Ip Security Camera,Outdoor Hd Wifi Ip Camera Product on Alibaba.com


found duplicate records
33
7203
Wholesale 2013 hot selling waterproof eva digital camera bag/cases - Alibaba.com


found duplicate records
72
4750
Deluxe Soft Padded Medium Bag For Digital Slr Camera Lens & Video Accessories Case - Buy Video Accessories Case,Deluxe Soft Padded Medium Bag,Medium Bag For Digital Slr Camera Lens Product on Alibaba.com


found duplicate records
125
1662
Camera Pouch - Buy Camera Pouch,Digital Camera Case,Camera Cover Product on Alibaba.com


found duplicate records
125
2836
Camera Pouch - Buy Camera Pouch,Digital Camera Case,Camera Cover Product on Alibaba.com


found duplicate records
125
3733
Camera Pouch - Buy Camera Pouch,Digital Camera Case,Camera Cover Product on Alibaba.com


found duplicate records
125
4661
C

found duplicate records
1646
3336
Croco Fashion Camera Bag - Buy Camera Bag,Fashion Camera Bag,New Design Digital Camera Bag Product on Alibaba.com


found duplicate records
1646
7556
Croco Fashion Camera Bag - Buy Camera Bag,Fashion Camera Bag,New Design Digital Camera Bag Product on Alibaba.com


found duplicate records
1660
2266
Croco Camera Case - Buy Camera Case,Digital Camera Case,Camera Bag Product on Alibaba.com


found duplicate records
1660
2960
Croco Camera Case - Buy Camera Case,Digital Camera Case,Camera Bag Product on Alibaba.com


found duplicate records
1660
3933
Croco Camera Case - Buy Camera Case,Digital Camera Case,Camera Bag Product on Alibaba.com


found duplicate records
1660
5709
Croco Camera Case - Buy Camera Case,Digital Camera Case,Camera Bag Product on Alibaba.com


found duplicate records
1660
6346
Croco Camera Case - Buy Camera Case,Digital Camera Case,Camera Bag Product on Alibaba.com


found duplicate records
1662
2836
Camera Pouch - Buy Camera Pouch,Digi

found duplicate records
4047
4757
Camera Case,Bag - Buy Camera Case,Digital Camera Bag,Compact Camera Case Product on Alibaba.com


found duplicate records
4047
7896
Camera Case,Bag - Buy Camera Case,Digital Camera Bag,Compact Camera Case Product on Alibaba.com


found duplicate records
4161
5173
2014 Hot Sale Fashionable Nylon Bag Case For Digital Camera - Buy Nylon,Digital Camera Nylon Case,Nylon Case For Samsung Digital Camera Product on Alibaba.com


found duplicate records
4161
5390
2014 Hot Sale Fashionable Nylon Bag Case For Digital Camera - Buy Nylon,Digital Camera Nylon Case,Nylon Case For Samsung Digital Camera Product on Alibaba.com


found duplicate records
4230
5371
Fashion Camera Pouch - Buy Camera Pouch,Digital Camera Bag,Compact Camera Case Product on Alibaba.com


found duplicate records
4230
6390
Fashion Camera Pouch - Buy Camera Pouch,Digital Camera Bag,Compact Camera Case Product on Alibaba.com


found duplicate records
4404
4508
2014 Genuine Leather Bag Case For Di

In [59]:
len(embedding_source2)

7892

In [75]:
len(set(titlesList_source2))

7892

In [77]:
import itertools

titlePairs = list(itertools.product(titlesList_source1, titlesList_source2))
len(embeddingPairs)

4552012

In [79]:
type(titlePairs[0])

tuple

In [80]:
type(titlePairs[0][1])

str

In [100]:
THRESHOLD = 0.9

def isMatching(firstTitle, secondTitle, threshold):
    firstEmbedding = embedding_source1[firstTitle].reshape(1,-1)
    secondEmbedding = embedding_source2[secondTitle].reshape(1,-1)
    similarityScore = cosine_similarity(firstEmbedding,secondEmbedding)[0][0]
    return similarityScore > threshold

In [122]:
# for testing purpose
sampledTitlePairs = random.sample(titlePairs, 2000)
len(sampledTitlePairs)

2000

In [123]:
# edit this line (replace sampleTitlePairs with titlePairs)
filteredTitlePairs = list(filter(lambda x: not isMatching(x[0],x[1],THRESHOLD), sampledTitlePairs))

In [124]:
len(filteredTitlePairs)

2000

In [115]:
firstTestTitle, secondTestTitle = filteredTitlePairs[0]
print(firstTestTitle)
print()
print(secondTestTitle)

B+W 52mm F-PRO UV MRC (010M) Brass Filters

Hikvision Security Camera Mini Dome Network Camera Ds-2cd2132-i - Buy Hikvision Security Camera,Hikvision Ip Network Camera,Hikvision Network Camera Product on Alibaba.com


In [125]:
isMatching(firstTestTitle, secondTestTitle, THRESHOLD)

False

In [135]:
csvData = []

# append matching records
for matchingRow in positives_samples:
    row = [matchingRow[0], matchingRow[1], matchingRow[2]]
    csvData.append(row)

print(len(csvData))
print()
print(csvData[0])

244

['Nikon Coolpix S6800 Digital Camera (Black) | UK Digital Cameras', 'NIKON COOLPIX S6800 RED 16MP 12X 32182', 1]


In [137]:
# append non-matching rows
for nonMatchingRow in filteredTitlePairs:
    row = [nonMatchingRow[0], nonMatchingRow[1], 0]
    csvData.append(row)
    
print(len(csvData))
print()
print(csvData[2243])

4244

['Nikon DF Kit with 50mm f/1.8G Lens Digital SLR Camera - Black', 'Korea Made Action Picture Video Camera Hd Quality - Buy Action Camera Hd 1080,Action Camera,Sport Camera Product on Alibaba.com', 0]


In [138]:
random.shuffle(csvData)

In [144]:
import csv 

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('dataset.csv', 'w') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['attributi_x', 'attributi_y', 'label'])
    for row in csvData:
        writer.writerow(row)